In [125]:
import pandas as pd

# Read csv files and get familiar with data

In [126]:
df_product = pd.read_csv('E://tiki_hometest/dim_product.csv')

In [127]:
df_event = pd.read_csv('E://tiki_hometest/events.csv')

In [128]:
df_revenue = pd.read_csv('E://tiki_hometest/revenue.csv')

In [129]:
df_product

,product_pseudo_id,category_pseudo_id
0,8362,0
1,2277,1802
2,2148,1802
3,107015,1802
4,101319,1802
...,...,...
138066,79450,1790
138067,24378,1790
138068,23558,1790
138069,103522,1790


In [130]:
df_event

,date_key,event_name,event_time,customer_pseudo_id,product_pseudo_id
0,2021-03-20,view,2021-03-20 10:57:42.903 UTC,256,2709.0
1,2021-03-20,view,2021-03-20 10:57:36.946 UTC,256,2709.0
2,2021-03-30,view,2021-03-30 02:02:45.694 UTC,256,48196.0
3,2021-04-16,view,2021-04-16 13:50:58.46 UTC,256,69379.0
4,2021-06-01,view,2021-06-01 04:19:58.135 UTC,256,133890.0
...,...,...,...,...,...
544564,2021-05-26,add_to_cart,2021-05-26 07:28:23.776 UTC,7935,78379.0
544565,2021-05-26,add_to_cart,2021-05-26 07:28:16.023 UTC,7935,78379.0
544566,2021-05-28,add_to_cart,2021-05-28 10:08:42.38 UTC,7935,107498.0
544567,2021-05-26,app_installed,2021-05-26 07:19:27 UTC,7935,NaN


In [131]:
df_revenue

,date_key,customer_pseudo_id,product_pseudo_id,order_pseudo_code,quantity,revenue
0,2021-03-02,3,82647,36,1,585200
1,2021-03-02,261,120179,15,1,91300
2,2021-03-02,9,132512,53,2,396000
3,2021-03-02,14,55040,42,1,124300
4,2021-03-02,526,93700,25,1,109670
...,...,...,...,...,...,...
17181,2021-03-01,754,9613,12890,1,72270
17182,2021-03-01,754,73142,12890,1,74470
17183,2021-03-01,1010,63560,12957,1,156200
17184,2021-03-01,5109,44114,12903,1,89265


# Step 1

In [132]:
# Remove rows contain `app_installed` and `register` values in the `event_name` column
# Add `category_pseudo_id` column by merging with `df_product`

df_event_2 = df_event[df_event['event_name'].isin(['search','view','add_to_cart','purchase'])].merge(df_product, how = 'left', on = 'product_pseudo_id')

In [133]:
# Convert values in `event_name` to numeric value to easily type

event_conversion = {'event_name':['search','view','add_to_cart','purchase'], 'step':[1,2,3,4]}
event_conversion = pd.DataFrame(event_conversion)
df_event_2 = df_event_2.merge(event_conversion, how = 'left', on = 'event_name')

In [134]:
# Create a list of customer ids
customer_list = df_event_2['customer_pseudo_id'].unique()

In [135]:
# Divide `df_event_2` into 4 small dataframes

df_1 = pd.DataFrame()
df_2 = pd.DataFrame()
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()

In [136]:
for customer_id in customer_list:
    event_info = df_event_2[df_event_2['customer_pseudo_id'] == customer_id]
    step_list = event_info['step'].unique()
    if 4 in step_list:
        df_4 = pd.concat([df_4, event_info])
    else:
        if 3 in step_list:
            df_3 = pd.concat([df_3, event_info])
        else:
            if 2 in step_list:
                df_2 = pd.concat([df_2, event_info])
            else:
                df_1 = pd.concat([df_1, event_info])

# Step 2

In [137]:
add_view = pd.DataFrame()
view_search = pd.DataFrame()

In [138]:
buy_intent = pd.DataFrame() 
add_intent = pd.DataFrame()
view_intent = pd.DataFrame()
search_intent = pd.DataFrame()

# 1. Purchase / Add_to_cart

In [139]:
buy_search_ratio = []
for customer in df_4['customer_pseudo_id'].unique():
    event_info = df_4[df_4['customer_pseudo_id'] == customer]
    if 1 in event_info['step'].unique():
        buy_search_ratio.append(len(event_info[event_info['step'] == 4])/len(event_info[event_info['step'] == 1]))
buy_search_mean = sum(buy_search_ratio)/len(buy_search_ratio)

In [140]:
buy_view_ratio = []
for customer in df_4['customer_pseudo_id'].unique():
    event_info = df_4[df_4['customer_pseudo_id'] == customer]
    if 2 in event_info['step'].unique():
        buy_view_ratio.append(len(event_info[event_info['step'] == 4])/len(event_info[event_info['step'] == 2]))
buy_view_mean = sum(buy_view_ratio)/len(buy_view_ratio)

In [141]:
buy_add_ratio = []
for customer in df_4['customer_pseudo_id'].unique():
    event_info = df_4[df_4['customer_pseudo_id'] == customer]
    if 3 in event_info['step'].unique():
        buy_add_ratio.append(len(event_info[event_info['step'] == 4])/len(event_info[event_info['step'] == 3]))
buy_add_mean = sum(buy_add_ratio)/len(buy_add_ratio)

In [ ]:
for customer in df_4['customer_pseudo_id'].unique():
    event_info = df_4[df_4['customer_pseudo_id'] == customer]
    step_list = event_info['step'].unique()
    buy_event = event_info[event_info['step'] == 4]
    add_event = event_info[event_info['step'] == 3]
    view_event = event_info[event_info['step'] == 2]
    search_event = event_info[event_info['step'] == 1]
    if 3 not in step_list:
        if 2 not in step_list:
            if 1 not in step_list:
                buy_intent = pd.concat([buy_intent,event_info])
            else:
                if len(buy_event)/len(search_event) >= buy_search_mean:
                    buy_intent = pd.concat([buy_intent,event_info])
                else:
                    search_intent = pd.concat([search_intent,event_info])
        else:
            if len(buy_event)/len(view_event) >= buy_view_mean:
                buy_intent = pd.concat([buy_intent,event_info])
            else:
                if 1 not in step_list:
                    view_intent = pd.concat([view_intent,event_info])
                else:
                    view_search = pd.concat([view_search,event_info])
    else:
        if len(buy_event)/len(add_event) >= buy_add_mean:
            buy_intent = pd.concat([buy_intent,event_info])
        else:
            add_view = pd.concat([add_view,event_info])

# 2. Add_to_cart / View

In [ ]:
add_search_ratio = []
for customer in df_event_2['customer_pseudo_id'].unique():
    event_info = df_event_2[df_event_2['customer_pseudo_id'] == customer]
    if (1 in event_info['step'].unique()) and (3 in event_info['step'].unique()):
        add_search_ratio.append(len(event_info[event_info['step'] == 3])/len(event_info[event_info['step'] == 1]))
add_search_mean = sum(add_search_ratio)/len(add_search_ratio)

In [ ]:
add_view_ratio = []
for customer in df_event_2['customer_pseudo_id'].unique():
    event_info = df_event_2[df_event_2['customer_pseudo_id'] == customer]
    if (2 in event_info['step'].unique()) and (3 in event_info['step'].unique()):
        add_view_ratio.append(len(event_info[event_info['step'] == 3])/len(event_info[event_info['step'] == 2]))
add_view_mean = sum(add_view_ratio)/len(add_view_ratio)

In [ ]:
add_view = pd.concat([add_view,df_3])
for customer in add_view['customer_pseudo_id'].unique():
    event_info = add_view[add_view['customer_pseudo_id'] == customer]
    step_list = event_info['step'].unique()
    add_event = event_info[event_info['step'] == 3]
    view_event = event_info[event_info['step'] == 2]
    search_event = event_info[event_info['step'] == 1]
    if 2 not in step_list:
        if 1 not in step_list:
            add_intent = pd.concat([add_intent,event_info])
        else:
            if len(add_event)/len(search_event) >= add_search_mean:
                add_intent = pd.concat([add_intent,event_info])
            else:
                search_intent = pd.concat([search_intent,event_info])
    else:
        if len(add_event)/len(view_event) >= add_view_mean:
            add_intent = pd.concat([add_intent,event_info])
        else:
            view_search = pd.concat([view_search,event_info])

# 3. View / Search

In [ ]:
view_search = pd.concat([view_search,df_2])

In [ ]:
view_search_ratio = []
for customer in df_event_2['customer_pseudo_id'].unique():
    event_info = df_event_2[df_event_2['customer_pseudo_id'] == customer]
    if (1 in event_info['step'].unique()) and (2 in event_info['step'].unique()):
        view_search_ratio.append(len(event_info[event_info['step'] == 2])/len(event_info[event_info['step'] == 1]))
view_search_mean = sum(view_search_ratio)/len(view_search_ratio)

In [ ]:
for customer in view_search['customer_pseudo_id'].unique():
    event_info = view_search[view_search['customer_pseudo_id'] == customer]
    step_list = event_info['step'].unique()
    view_event = event_info[event_info['step'] == 2]
    search_event = event_info[event_info['step'] == 1]
    if 1 not in step_list:
        view_intent = pd.concat([view_intent,event_info])
    else:
        if len(view_event)/len(search_event) >= view_search_mean:
            view_intent = pd.concat([view_intent,event_info])
        else:
            search_intent = pd.concat([search_intent,event_info])

In [ ]:
search_intent = pd.concat([search_intent,df_1])

In [ ]:
# Count number of customers in each purchase intention levels

purchase_intention_count = pd.DataFrame({'purchase_intention_level':['Awareness','Consideration','Conversion','Buy','Total'],
                                         'number_of_customers':[(search_intent['customer_pseudo_id'].nunique()),
                                                                (view_intent['customer_pseudo_id'].nunique()),
                                                                (add_intent['customer_pseudo_id'].nunique()),
                                                                (buy_intent['customer_pseudo_id'].nunique()),
                                                                (df_event_2['customer_pseudo_id'].nunique())]})

In [ ]:
purchase_intention_count

# Purchase intention of each customer by product categories in each intention level

In [ ]:
# Buy

buy_intent[buy_intent['event_name'] == 'purchase'][['customer_pseudo_id','category_pseudo_id','event_name']].groupby(['customer_pseudo_id','category_pseudo_id']).count().sort_values('event_name', ascending = False)

In [ ]:
# Conversion

add_intent[add_intent['event_name'] == 'add_to_cart'][['customer_pseudo_id','category_pseudo_id','event_name']].groupby(['customer_pseudo_id','category_pseudo_id']).count().sort_values('event_name', ascending = False)

In [ ]:
# Consideration

view_intent[view_intent['event_name'] == 'view'][['customer_pseudo_id','category_pseudo_id','event_name']].groupby(['customer_pseudo_id','category_pseudo_id']).count().sort_values('event_name', ascending = False)